[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Lvb9jrpDE52l8ZZWXeBoxbjXR5eHc6ft?usp=sharing)

# Portkey Agent Integration Cookbook

## What is [Portkey AI](https://portkey.ai)?


Portkey AI is a production infrastructure layer that makes your AI agents production-ready with just a few lines of code changes. It sits between your agents and LLM providers, adding powerful capabilities like routing, monitoring, and reliability features.


Portkey adds 4 core production capabilities to any agent:
1. Routing to 200+ LLMs
2. Making each LLM call more robust
3. Full-stack tracing & cost, performance analytics
4. Real-time guardrails to enforce behavior


# Framework Integrations

E2E examples of major Agent Frameworks:
1. Autogen
2. CrewAI
4. Langchain Agents
5. LangGraph Agents
6. OpenAI Swarm and more

## Key Features
Portkey offers a range of advanced features to enhance your agents. Here’s an overview

| Feature | Description |
|---------|-------------|
| 🌐  Multi-LLM Integration | Access 200+ LLMs with simple configuration changes |
| 🛡️ Enhanced Reliability | Implement fallbacks, load balancing, retries, and much more |
| 📊 Advanced Metrics | Track costs, tokens, latency, and 40+ custom metrics effortlessly |
| 🔍 Detailed Traces and Logs | Gain insights into every agent action and decision |
| 🚧 Guardrails Enforce agent behavior with real-time checks on inputs and outputs |
| 🔄 Continuous Optimization | Capture user feedback for ongoing agent improvements |
| 💾 Smart Caching | Reduce costs and latency with built-in caching mechanisms |
| 🔐 Enterprise-Grade Security | Set budget limits and implement fine-grained access controls |








Here’s the output from your Agent’s run on Portkey's dashboard

<img src="https://github.com/siddharthsambharia-portkey/Portkey-Product-Images/blob/main/Portkey-Dashboard.png?raw=true" alt="Portkey Dashboard" />



## Prerequisites

Before starting:
- Get your Portkey API key from [Portkey Dashboard](https://app.portkey.ai)
- Install the Portkey SDK: `pip install portkey-ai`
- Have your LLM provider API keys ready





## **Autogen**

In this example, we're creating a development team of agents to build a game:

A Product Manager to define requirements
A Coder to implement the game
A User Proxy to facilitate interaction

In [ ]:
!pip install -qU pyautogen portkey-ai

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from google.colab import userdata

# Import the portkey library to fetch helper functions
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders

gpt3_config = [
    {
        "api_key": userdata.get("OPENAI_API_KEY"),
        "model": "gpt-3.5-turbo",
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai",
        "default_headers": createHeaders(
            api_key = userdata.get("PORTKEY_API_KEY"),
            provider = "openai",
        )
    }
]


gpt4o_config = [
    {
        "api_key": userdata.get("OPENAI_API_KEY"),
        "model": "gpt-4o",
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai",
        "default_headers": createHeaders(
            api_key = userdata.get("PORTKEY_API_KEY"),
            provider = "openai",
        )
    }
]

llama3_config = [
    {
        "api_key": userdata.get("GROQ_API_KEY"),
        "model": "llama3-70b-8192",
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai", # Portkey conforms to the openai api_type
        "default_headers": createHeaders(
            api_key = userdata.get("PORTKEY_API_KEY"),
            provider = "groq",
        )
    }
]

In [ ]:
import autogen

# Create user proxy agent, coder, product manager
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin who will give the idea and run the code provided by Coder.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="ALWAYS",
)
coder = autogen.AssistantAgent(
    name="Coder",
    system_message = "You are talented Python developer who is good at developing games. You closely work with Product Manager."
    llm_config={"config_list": llama3_config},
)
pm = autogen.AssistantAgent(
    name="product_manager",
    system_message="You will help break down the initial idea into a well scoped requirement for the coder; Do not involve in future conversations or error fixing",
    llm_config={"config_list": gpt3_config},
)

# Create groupchat
groupchat = autogen.GroupChat(
    agents=[user_proxy, coder, pm], messages=[])
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={"config_list": gpt4o_config})

# Start the conversation
user_proxy.initiate_chat(
    manager, message="Build a classic & basic pong game with 2 players in python")

## **CrewAI**

Here we're building a research and analysis team that works together to analyze a dataset:

A Data Analyst to crunch numbers
A Research Writer to create reports
A Reviewer to validate findings

In [ ]:
!pip install -qU crewai portkey-ai

In [ ]:
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL
from google.colab import userdata
import getpass

portkey_api_key = getpass.getpass("Enter your Portkey API Key: ")
openai_key = getpass.getpass("Enter your OpenAI API Key: ")
deepinfra_key = getpass.getpass("Enter your Deepinfra API Key: ")

gpt_3 = ChatOpenAI(
    api_key=openai_key,
    base_url=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(api_key=portkey_api_key,provider="openai")
)

llama3 = ChatOpenAI(
    api_key = deepinfra_key,
    base_url=PORTKEY_GATEWAY_URL,
    model = "meta-llama/Meta-Llama-3-70B-Instruct",
    default_headers=createHeaders(api_key=portkey_api_key,provider="deepinfra")
)

In [ ]:
from crewai import Agent, Task, Crew, Process

# Define your agents with roles and goals
product_manager = Agent(
    role='Product Manager',
    goal='Define requirements for a software product',
    backstory="You are an experienced Product Manager skilled in defining clear and concise requirements.",
    llm = gpt_3
)

coder = Agent(
    role='Software Developer',
    goal='Develop software based on the provided requirements',
    backstory="You are a skilled software developer proficient in coding robust and efficient applications.",
    llm = llama3
)

# Create tasks for your agents
task1 = Task(
    description="Define the key requirements and features for a classic ping pong game. Be specific and concise.",
    expected_output="A clear and concise list of requirements for the ping pong game",
    agent=product_manager
)

task2 = Task(
    description="Based on the provided requirements, develop the code for the classic ping pong game. Focus on gameplay mechanics and a simple user interface.",
    expected_output="Complete code for the ping pong game",
    agent=coder
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[product_manager, coder],
    tasks=[task1, task2],
    verbose=1,
)

# Get your crew to work!
result = crew.kickoff()
print("######################")
print(result)

## **Langchain Agents**

In [ ]:
!pip install -qU langchain_openai portkey-ai langchain langchain_community duckduckgo-search langchainhub

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL, Portke

from langchain.prompts import PromptTemplate
from portkey_ai import Portkey

# Initialize the DuckDuckGo search tool
search_tool = DuckDuckGoSearchResults(max_results=5)

# Ensure tools is a list containing the search tool
tools = [search_tool]


client = Portkey(
    api_key="PORTKEY_API_KEY",  #Enter Your PORTKEY API KEY
)

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")


llm = ChatOpenAI(
    api_key="OPENAI_API_KEY", #YOUR OPENAI API KEY
    base_url=PORTKEY_GATEWAY_URL,
    model="gpt-3.5-turbo",
    default_headers=createHeaders(api_key="PORTKEY_API_KEY", provider="openai") #YOUR PORTKEY API KEY
)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## **Langraph Agents**

In this example, we're building a conversational agent with state management:
- Maintains conversation context
- Handles multi-turn dialogues
- Provides streaming responses
- Demonstrates graph-based agent architecture

Ideal for:
- Customer service chatbots
- Interactive assistants
- Multi-step conversations


In [ ]:
pip install -U langgraph langchain_openai portkey-ai


In [ ]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL

from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = ChatOpenAI(
    api_key="OpenAI_API_Key",
    base_url=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        provider="openai",
        api_key="PORTKEY_API_KEY",
    )
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break


## **LlamaIndex agents**

In [ ]:
pip install -qU llama-agents llama-index portkey-ai


In [ ]:
from llama_index.llms.openai import OpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders

gpt_4o_config = {
    "provider": "openai", #Use the provider of choice
    "api_key": "YOUR_OPENAI_KEY,
    "override_params": { "model":"gpt-4o" }
}

gpt_4o = OpenAI(
    api_base=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        api_key=userdata.get('PORTKEY_API_KEY'),
        config=gpt_4o_config
    )
)


In [ ]:
from llama_index.llms.openai import OpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders
from google.colab import userdata


gpt_4o_config = {
    "provider": "openai",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "override_params": { "model":"gpt-4o" }
}

gpt_4o = OpenAI(
    api_base=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        api_key=userdata.get('PORTKEY_API_KEY'),
        config=gpt_4o_config
    )
)

llama3_groq_config = {
    "provider": "groq",
    "api_key": userdata.get('GROQ_API_KEY'),
    "override_params": {"model":"llama3-70b-8192"}
}

llama3 = OpenAI(
    api_base=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        api_key=userdata.get('PORTKEY_API_KEY'),
        config=llama3_groq_config
    )
)

# CREATING THE AGENT
from llama_agents import (
    AgentService,
    AgentOrchestrator,
    ControlPlaneServer,
    SimpleMessageQueue,
)

from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI


# create an agent
def get_the_secret_fact() -> str:
    """Returns the secret fact."""
    return "The secret fact is: A baby llama is called a 'Cria'."


tool = FunctionTool.from_defaults(fn=get_the_secret_fact)

agent1 = ReActAgent.from_tools([tool], llm=llama3)
agent2 = ReActAgent.from_tools([], llm=llama3)

# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=gpt_4o),
)

agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Useful for getting the secret fact.",
    service_name="secret_fact_agent"
)

agent_server_2 = AgentService(
    agent=agent2,
    message_queue=message_queue,
    description="Useful for getting random dumb facts.",
    service_name="dumb_fact_agent"
)

In [ ]:
from llama_agents import LocalLauncher

# launch it
launcher = LocalLauncher(
    [agent_server_1, agent_server_2],
    control_plane,
    message_queue,
)
result = launcher.launch_single("What is the secret fact?")

print(f"Result: {result}")

## **OpenAI Swarm**

Here's a simple weather agent built using OpenAI Swarm

In [ ]:
from swarm import Swarm, Agent
from portkey_ai import Portkey

portkey = Portkey(
    api_key=userdata.get('PORTKEY_API_KEY'), # defaults to os.environ.get("OPENAI_API_KEY")
    Authorization="YOUR_OPENAI_API_KEY",
    provider="openai
    )

client = Swarm(client=portkey)


def get_weather(location) -> str:
    return "{'temp':67, 'unit':'F'}"


agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather],
)

messages = [{"role": "user", "content": "What's the weather in NYC?"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

## Portkey Features

### Interoperability

Easily switch between **200+ LLMs** by changing the configuration in your client,. In this example we are using Virtual Keys to sseamlessly change our LLM client. Portkey offers a virtual key system that allows you to securely store your Language Model (LLM) API keys in their vault. This system uses a unique virtual identifier to simplify API key management.


#### Example: Switching from OpenAI to Azure OpenAI

```python
portkey = Portkey(
    api_key=userdata.get('PORTKEY_API_KEY'), # defaults to os.environ.get("OPENAI_API_KEY")
    virtual_key="YOUR_OPENAI_VIRTUAL_KEY",
    )

# Changing the virtual-key to shift to shift to Azure

portkey = Portkey(
    api_key=userdata.get('PORTKEY_API_KEY'), # defaults to os.environ.get("OPENAI_API_KEY")
    virtual_key="YOUR_AZURE_VIRTUAL_KEY",
    )
```


### Reliability

Implement fallbacks, load balancing, and automatic retries to make your agents more resilient.

```python
{
  "strategy": {
    "mode": "fallback" # Options: "loadbalance" or "fallback"
  },
  "targets": [
    {
      "provider": "openai",
      "api_key": "openai-api-key",
      "override_params": {
        "top_k": "0.4",
        "max_tokens": "100"
      }
    },
    {
      "provider": "anthropic",
      "api_key": "anthropic-api-key",
      "override_params": {
        "top_p": "0.6",
        "model": "claude-3-5-sonnet-20240620"
      }
    }
  ]
}
```
Learn more about [Portkey Config object here](https://docs.portkey.ai/docs/product/ai-gateway-streamline-llm-integrations/configs).
Be Careful to Load-Balance/Fallback to providers that don't support tool calling when the request contains a function call.
### Metrics

Agent runs are complex. Portkey automatically logs **40+ comprehensive metrics** for your AI agents, including cost, tokens used, latency, etc. Whether you need a broad overview or granular insights into your agent runs, Portkey's customizable filters provide the metrics you need.

<details>
  <summary><b>Portkey's Observability Dashboard</b></summary>
<img src=https://github.com/siddharthsambharia-portkey/Portkey-Product-Images/blob/main/Portkey-Dashboard.png?raw=true width=70%" alt="Portkey Dashboard" />
</details>

### Comprehensive Logging

Access detailed logs and traces of agent activities, function calls, and errors. Filter logs based on multiple parameters for in-depth analysis.

<details>
  <summary><b>Traces</b></summary>
  <img src="https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/Portkey-Traces.png" alt="Portkey Logging Interface" width=70% />
</details>

<details>
  <summary><b>Logs</b></summary>
  <img src="https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/Portkey-Logs.png" alt="Portkey Metrics Visualization" width=70% />
</details>

### Guardrails
AI agents, while powerful, can sometimes produce unexpected or undesired outputs. Portkey's Guardrails feature helps enforce agent behavior in real-time, ensuring your agents operate within specified parameters. Verify both the **inputs** to and *outputs* from your agents to ensure they adhere to specified formats and content guidelines. Learn more about Portkey's Guardrails [here](https://docs.portkey.ai/product/guardrails)

### Continuous Improvement

Capture qualitative and quantitative user feedback on your requests to continuously enhance your agent performance.

### Caching

Reduce costs and latency with Portkey's built-in caching system.

```python
portkey_config = {
 "cache": {
    "mode": "semantic"  # Options: "simple" or "semantic"
 }
}
```

### Security and Compliance

Set budget limits on provider API keys and implement fine-grained user roles and permissions for both your application and the Portkey APIs.

## Additional Resources

- [📘 Portkey Documentation](http://docs.portkey.ai/?utm_source=NirDiamant&utm_medium=github&utm_campaign=external-integration)
- [🐦 Twitter](https://twitter.com/portkeyai/)
- [💬 Discord Community](https://discord.gg/JHPt4C7r)
- [📊 Portkey App](https://app.portkey.ai/?utm_source=NirDiamant&utm_medium=github&utm_campaign=external-integration)

For more information on using these features and setting up your Config, please refer to the [Portkey documentation](https://docs.portkey.ai/?utm_source=NirDiamant&utm_medium=github&utm_campaign=external-integration).